In [1]:
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import pandas as pd
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math

from ipywidgets import interact
from PIL import Image
from sklearn import preprocessing
from scipy.stats import pearsonr # 科学技術計算ライブラリ
from sklearn.preprocessing import OneHotEncoder

In [2]:
net = models.resnet18(pretrained=True)
#net = models.vgg16(pretrained=True)

num_ftrs = net.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
net.fc = nn.Linear(num_ftrs, 7)

print(num_ftrs)

512


In [5]:
net = models.resnet18(pretrained=True)
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
male_female_dataset = pd.read_csv("male_female_dataset.csv",index_col=0)
male_female_dataset

,path,word1,word2,word3,word4,word5,word6,word7,attribute
0,11n24y (10).jpg,0.592593,0.608446,0.551515,0.193548,0.655914,0.703704,0.375000,0
1,11n24y (15).jpg,0.476190,0.538745,0.418182,0.156682,0.686636,0.547619,0.535714,0
2,11n24y (20).jpg,0.537037,0.551046,0.636364,0.279570,0.698925,0.638889,0.437500,0
3,11n24y (22).jpg,0.481481,0.723247,0.318182,0.322581,0.763441,0.722222,0.406250,0
4,11n24y (24).jpg,0.505051,0.488427,0.242975,0.357771,0.671554,0.636364,0.443182,0
...,...,...,...,...,...,...,...,...,...
1992,yumben (57).jpg,0.518519,0.587500,0.633333,0.533333,0.700000,0.600000,0.520000,1
1993,yumben (58).jpg,0.444444,0.578125,0.791667,0.333333,0.875000,0.636364,0.600000,1
1994,yumben (59).jpg,0.444444,0.531250,0.750000,0.375000,0.416667,0.590909,0.500000,1
1995,yumben (60).jpg,0.768519,0.484375,0.541667,0.708333,0.541667,0.590909,0.700000,1


In [11]:
attribute = male_female_dataset["attribute"].values

attribute_enc = preprocessing.LabelEncoder().fit_transform(attribute).reshape(-1,1)
attribute_enc2 = OneHotEncoder().fit_transform(attribute_enc).toarray()
attribute_enc2 = torch.Tensor(attribute_enc2)
attribute_enc2

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [14]:
attribute_enc2[1000]

tensor([0., 1.])

In [22]:
x = torch.randn(20)
x

tensor([-0.1208,  0.7789, -0.2960, -1.0344, -0.1148,  0.6091, -0.0580,  0.0865,
         0.0519, -0.1401, -0.0440, -0.5021, -1.3064, -0.2854,  0.5661, -0.6892,
         1.2622,  0.9075, -2.7065, -2.1075])

In [27]:
torch.cat((x,attribute_enc2[1000]), 0)

tensor([-0.1208,  0.7789, -0.2960, -1.0344, -0.1148,  0.6091, -0.0580,  0.0865,
         0.0519, -0.1401, -0.0440, -0.5021, -1.3064, -0.2854,  0.5661, -0.6892,
         1.2622,  0.9075, -2.7065, -2.1075,  0.0000,  1.0000])

In [28]:
x.shape

torch.Size([20])

## 結果比較

In [16]:
log_path = '/home/taki/bento_conditional/log/'

conditional_flg = [0,1]
optim_flg = ["Adam"]
batch_size = [32]
in_w = [640]
lr = [0.001, 0.0001, 0.00001]
weight_decay = [0.0001, 0.00001, 0.000001]
seed = [0]

path_list = []

for i in conditional_flg:
    for j in optim_flg:
        for k in batch_size:
            for l in in_w:
                for m in lr:
                    for n in weight_decay:
                        for o in seed:
                            param_folder = f'conditional_{i}_optim{j}_batch{k}_w,h{l}_lr{m}_wDecay{n}_seed{o}'
                            path_list.append(param_folder)
path_list

['conditional_0_optimAdam_batch32_w,h640_lr0.001_wDecay0.0001_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr0.001_wDecay1e-05_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr0.001_wDecay1e-06_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr0.0001_wDecay0.0001_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr0.0001_wDecay1e-05_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr0.0001_wDecay1e-06_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr1e-05_wDecay0.0001_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr1e-05_wDecay1e-05_seed0',
 'conditional_0_optimAdam_batch32_w,h640_lr1e-05_wDecay1e-06_seed0',
 'conditional_1_optimAdam_batch32_w,h640_lr0.001_wDecay0.0001_seed0',
 'conditional_1_optimAdam_batch32_w,h640_lr0.001_wDecay1e-05_seed0',
 'conditional_1_optimAdam_batch32_w,h640_lr0.001_wDecay1e-06_seed0',
 'conditional_1_optimAdam_batch32_w,h640_lr0.0001_wDecay0.0001_seed0',
 'conditional_1_optimAdam_batch32_w,h640_lr0.0001_wDecay1e-05_seed0',
 'conditional_1_optimAda

In [38]:

log_path = '/home/taki/bento_conditional/log/'

conditional_flg = [0,1]
optim_flg = ["Adam"]
batch_size = [32]
in_w = [640]
lr = [0.001, 0.0001, 0.00001]
weight_decay = [0.0001, 0.00001, 0.000001]
seed = [0]

path_list = []

for i in conditional_flg:
    for j in optim_flg:
        for k in batch_size:
            for l in in_w:
                for m in lr:
                    for n in weight_decay:
                        for o in seed:
                            param_folder = f'conditional_{i}_optim{j}_batch{k}_w,h{l}_lr{m}_wDecay{n}_seed{o}'
                            path_list.append(param_folder)


col = ["color","healthy","satisfaction","uniqueness","ease of eating","appropriate amount","not collapse"]
list_for_result = []

for path_i in path_list:

    dir = os.path.join(log_path, path_i)
    
    out_df_list   = []
    score_df_list = []



    for cv_num in range(1):
        out_dir   = f"/closs_valid{cv_num}/test_out_df.csv"
        score_dir = f"/closs_valid{cv_num}/test_score_df.csv"
        out_dir   = dir + out_dir
        score_dir = dir + score_dir

        out_df = pd.read_csv(out_dir)
        score_df = pd.read_csv(score_dir)

        out_df_list.append(out_df)
        score_df_list.append(score_df)
    

    out_df = out_df_list[0]    # concatするために最初に入れとく
    score_df = score_df_list[0]
    for i in range(len(out_df_list)-1):
        out_df = pd.concat([out_df,out_df_list[i+1]])
        score_df = pd.concat([score_df,score_df_list[i+1]])

    out_df = out_df.drop("Unnamed: 0",axis=1)
    score_df = score_df.drop("Unnamed: 0",axis=1)

    col_list = {}
    for i in range(len(col)):
        col_list[f"{i}"] = col[i]
    out_df = out_df.rename(columns=col_list)
    out_df = out_df.reset_index(drop=True)
    score_df = score_df.rename(columns=col_list)
    score_df = score_df.reset_index(drop=True)

    temp = []
    for col_i in col:
        a, b = pearsonr(out_df[col_i], score_df[col_i])
        temp.append(a)
    list_for_result.append(temp)

corr_df = pd.DataFrame(list_for_result, columns=col)
corr_df.insert(0,"path",path_list)


In [45]:
corr_df["mean"] = corr_df.mean(axis='columns')
corr_df

,path,color,healthy,satisfaction,uniqueness,ease of eating,appropriate amount,not collapse,mean
0,"conditional_0_optimAdam_batch32_w,h640_lr0.001...",0.612254,0.548191,0.482699,0.499206,0.577935,0.397860,0.188001,0.472307
1,"conditional_0_optimAdam_batch32_w,h640_lr0.001...",0.691891,0.599202,0.455461,0.527813,0.562965,0.331421,0.191018,0.479967
2,"conditional_0_optimAdam_batch32_w,h640_lr0.001...",0.669874,0.613787,0.480133,0.466737,0.597562,0.298704,0.183966,0.472966
3,"conditional_0_optimAdam_batch32_w,h640_lr0.000...",0.648190,0.598958,0.470617,0.469181,0.635413,0.357571,0.168279,0.478316
4,"conditional_0_optimAdam_batch32_w,h640_lr0.000...",0.654063,0.617403,0.541578,0.407535,0.632916,0.295332,-0.015349,0.447640
5,"conditional_0_optimAdam_batch32_w,h640_lr0.000...",0.670676,0.616933,0.465412,0.493743,0.665854,0.309405,0.086039,0.472580
6,"conditional_0_optimAdam_batch32_w,h640_lr1e-05...",0.724447,0.640180,0.464248,0.499469,0.595872,0.419365,0.308835,0.521774
7,"conditional_0_optimAdam_batch32_w,h640_lr1e-05...",0.697755,0.598850,0.536373,0.478169,0.572643,0.336139,0.332692,0.507517
8,"conditional_0_optimAdam_batch32_w,h640_lr1e-05...",0.656251,0.583390,0.539526,0.492034,0.618805,0.383930,0.135568,0.487072
9,"conditional_1_optimAdam_batch32_w,h640_lr0.001...",0.684221,0.638300,0.648318,0.548301,0.647739,0.316641,0.275649,0.537024


In [48]:
corr_df.iloc[6]

path                  conditional_0_optimAdam_batch32_w,h640_lr1e-05...
color                                                          0.724447
healthy                                                         0.64018
satisfaction                                                   0.464248
uniqueness                                                     0.499469
ease of eating                                                 0.595872
appropriate amount                                             0.419365
not collapse                                                   0.308835
mean                                                           0.521774
Name: 6, dtype: object

In [46]:
path_list[6]

'conditional_0_optimAdam_batch32_w,h640_lr1e-05_wDecay0.0001_seed0'

In [47]:
path_list[15]

'conditional_1_optimAdam_batch32_w,h640_lr1e-05_wDecay0.0001_seed0'

In [20]:
list_for_result = []
temp = []
for col_i in col:
    a, b = pearsonr(out_df[col_i], score_df[col_i])
    temp.append(a)
list_for_result.append(temp)

colorの相関係数: 0.6122544854679548
healthyの相関係数: 0.5481914650400361
satisfactionの相関係数: 0.4826986379325642
uniquenessの相関係数: 0.4992058491849208
ease of eatingの相関係数: 0.5779353347504297
appropriate amountの相関係数: 0.39785978430856217
not collapseの相関係数: 0.18800106866820024


In [36]:
test = pd.DataFrame(list_for_result, columns=col)
test.insert(0,"path",path_list)
test

,path,color,healthy,satisfaction,uniqueness,ease of eating,appropriate amount,not collapse
0,"conditional_0_optimAdam_batch32_w,h640_lr0.001...",0.612254,0.548191,0.482699,0.499206,0.577935,0.397860,0.188001
1,"conditional_0_optimAdam_batch32_w,h640_lr0.001...",0.691891,0.599202,0.455461,0.527813,0.562965,0.331421,0.191018
2,"conditional_0_optimAdam_batch32_w,h640_lr0.001...",0.669874,0.613787,0.480133,0.466737,0.597562,0.298704,0.183966
3,"conditional_0_optimAdam_batch32_w,h640_lr0.000...",0.648190,0.598958,0.470617,0.469181,0.635413,0.357571,0.168279
4,"conditional_0_optimAdam_batch32_w,h640_lr0.000...",0.654063,0.617403,0.541578,0.407535,0.632916,0.295332,-0.015349
5,"conditional_0_optimAdam_batch32_w,h640_lr0.000...",0.670676,0.616933,0.465412,0.493743,0.665854,0.309405,0.086039
6,"conditional_0_optimAdam_batch32_w,h640_lr1e-05...",0.724447,0.640180,0.464248,0.499469,0.595872,0.419365,0.308835
7,"conditional_0_optimAdam_batch32_w,h640_lr1e-05...",0.697755,0.598850,0.536373,0.478169,0.572643,0.336139,0.332692
8,"conditional_0_optimAdam_batch32_w,h640_lr1e-05...",0.656251,0.583390,0.539526,0.492034,0.618805,0.383930,0.135568
9,"conditional_1_optimAdam_batch32_w,h640_lr0.001...",0.684221,0.638300,0.648318,0.548301,0.647739,0.316641,0.275649


In [22]:
data1

[0.6122544854679548,
 0.5481914650400361,
 0.4826986379325642,
 0.4992058491849208,
 0.5779353347504297,
 0.39785978430856217,
 0.18800106866820024]

In [8]:
a = {'色鮮やか': 0.6877423294297559, '体にいい, 体によさそう': 0.5751277366371776, 'ボリューム感がある': 0.6140329570877112, '珍しい': 0.5068510574616384, '一口サイズ': 0.5750980309550522, '早く食べれる': 0.31644432419330165, '場所が固まっている': 0.17621548394446418}

sum(a.values()) / len(a)

0.4930731313870145

In [7]:
a.values().mean()

AttributeError: 'dict_values' object has no attribute 'mean'

In [15]:
a = [1,2,3,4,5]
np.mean(a)

3.0

In [13]:
conditional_0 = {'色鮮やか': 0.7677892857446049, '体にいい, 体によさそう': 0.537122571679345, 'ボリューム感がある': 0.7025871277291539, '珍しい': 0.5425434509470823, '一口サイズ': 0.5935909781429091, '早く食べれる': 0.394609429972449, '場所が固まっている': 0.29588142587195304}
conditional_1 = {'色鮮やか': 0.7090677551858803, '体にいい, 体によさそう': 0.5960370162658636, 'ボリューム感がある': 0.6868293615592453, '珍しい': 0.5409549956369476, '一口サイズ': 0.6877761607205588, '早く食べれる': 0.35098008093874605, '場所が固まっている': 0.16371616317379623}
conditional_2 = {'色鮮やか': 0.687708880829605, '体にいい, 体によさそう': 0.5786696651960004, 'ボリューム感がある': 0.6748885828978286, '珍しい': 0.3912003715194349, '一口サイズ': 0.5680932984918483, '早く食べれる': 0.44745090651440583, '場所が固まっている': 0.2833968381140597}
conditional_3 = {'色鮮やか': 0.6764731444976682, '体にいい, 体によさそう': 0.5372979625880193, 'ボリューム感がある': 0.7019885580845195, '珍しい': 0.5957348999668446, '一口サイズ': 0.6710563745759611, '早く食べれる': 0.47653217793521196, '場所が固まっている': 0.16719313599818322}
conditional_4 = {'色鮮やか': 0.6981074646371574, '体にいい, 体によさそう': 0.5846908250871765, 'ボリューム感がある': 0.757951453339833, '珍しい': 0.5533596178136105, '一口サイズ': 0.5751862277115275, '早く食べれる': 0.4577007321770341, '場所が固まっている': 0.20593312938397765}

conditional_0_df =  pd.DataFrame(conditional_0.values(),index=conditional_0.keys()).T
conditional_1_df =  pd.DataFrame(conditional_1.values(),index=conditional_1.keys()).T
conditional_2_df =  pd.DataFrame(conditional_2.values(),index=conditional_2.keys()).T
conditional_3_df =  pd.DataFrame(conditional_3.values(),index=conditional_3.keys()).T
conditional_4_df =  pd.DataFrame(conditional_4.values(),index=conditional_4.keys()).T

conditional_df = pd.concat([conditional_0_df,conditional_1_df])
conditional_df = pd.concat([conditional_df,conditional_2_df])
conditional_df = pd.concat([conditional_df,conditional_3_df])
conditional_df = pd.concat([conditional_df,conditional_4_df])
conditional_df

,色鮮やか,"体にいい, 体によさそう",ボリューム感がある,珍しい,一口サイズ,早く食べれる,場所が固まっている
0,0.767789,0.537123,0.702587,0.542543,0.593591,0.394609,0.295881
0,0.709068,0.596037,0.686829,0.540955,0.687776,0.350980,0.163716
0,0.687709,0.578670,0.674889,0.391200,0.568093,0.447451,0.283397
0,0.676473,0.537298,0.701989,0.595735,0.671056,0.476532,0.167193
0,0.698107,0.584691,0.757951,0.553360,0.575186,0.457701,0.205933


In [28]:
for i in range(len(conditional_df.columns)):
    keys = conditional_df.columns[i]
    print(f"{keys} : {conditional_df[keys].mean()} ± {conditional_df[keys].var()}")
    

色鮮やか : 0.7078293061789831 ± 0.0012698218750103747
体にいい, 体によさそう : 0.566763608163281 ± 0.0007667217589130214
ボリューム感がある : 0.7048490167221161 ± 0.0010138756768252038
珍しい : 0.524758667176784 ± 0.006068019481660179
一口サイズ : 0.6191406079285608 ± 0.0031491712142063337
早く食べれる : 0.4254546655075694 ± 0.002657612002948562
場所が固まっている : 0.22322413850839395 ± 0.003969871638031868


In [15]:
non_conditional_0 = {'色鮮やか': 0.726873288244427, '体にいい, 体によさそう': 0.597651861674547, 'ボリューム感がある': 0.6197739502822471, '珍しい': 0.5578098783046466, '一口サイズ': 0.6736700460644592, '早く食べれる': 0.4400704702693983, '場所が固まっている': 0.2048016786940119}
non_conditional_1 = {'色鮮やか': 0.6999414536903755, '体にいい, 体によさそう': 0.6422379483898087, 'ボリューム感がある': 0.590962499576395, '珍しい': 0.5209846295952543, '一口サイズ': 0.6773439399539172, '早く食べれる': 0.3687458843233665, '場所が固まっている': 0.15917344791478233}
non_conditional_2 = {'色鮮やか': 0.6746619890635337, '体にいい, 体によさそう': 0.6102541841606777, 'ボリューム感がある': 0.5404023009048637, '珍しい': 0.4160694804504411, '一口サイズ': 0.6166465446540186, '早く食べれる': 0.4365477828281543, '場所が固まっている': 0.28370543618304456}
non_conditional_3 = {'色鮮やか': 0.6855865213818658, '体にいい, 体によさそう': 0.6056930763538896, 'ボリューム感がある': 0.5897013668117755, '珍しい': 0.5906607293235321, '一口サイズ': 0.6570921189844674, '早く食べれる': 0.45680021340440913, '場所が固まっている': 0.18410765979381505}
non_conditional_4 = {'色鮮やか': 0.6997008217544625, '体にいい, 体によさそう': 0.6117522906615752, 'ボリューム感がある': 0.6000283685239177, '珍しい': 0.5681533192412943, '一口サイズ': 0.5538134475054226, '早く食べれる': 0.4224387257265654, '場所が固まっている': 0.18178373424561392}

non_conditional_0_df =  pd.DataFrame(non_conditional_0.values(),index=non_conditional_0.keys()).T
non_conditional_1_df =  pd.DataFrame(non_conditional_1.values(),index=non_conditional_1.keys()).T
non_conditional_2_df =  pd.DataFrame(non_conditional_2.values(),index=non_conditional_2.keys()).T
non_conditional_3_df =  pd.DataFrame(non_conditional_3.values(),index=non_conditional_3.keys()).T
non_conditional_4_df =  pd.DataFrame(non_conditional_4.values(),index=non_conditional_4.keys()).T

non_conditional_df = pd.concat([non_conditional_0_df,non_conditional_1_df])
non_conditional_df = pd.concat([non_conditional_df,non_conditional_2_df])
non_conditional_df = pd.concat([non_conditional_df,non_conditional_3_df])
non_conditional_df = pd.concat([non_conditional_df,non_conditional_4_df])
non_conditional_df

,色鮮やか,"体にいい, 体によさそう",ボリューム感がある,珍しい,一口サイズ,早く食べれる,場所が固まっている
0,0.726873,0.597652,0.619774,0.557810,0.673670,0.440070,0.204802
0,0.699941,0.642238,0.590962,0.520985,0.677344,0.368746,0.159173
0,0.674662,0.610254,0.540402,0.416069,0.616647,0.436548,0.283705
0,0.685587,0.605693,0.589701,0.590661,0.657092,0.456800,0.184108
0,0.699701,0.611752,0.600028,0.568153,0.553813,0.422439,0.181784


In [29]:
for i in range(len(non_conditional_df.columns)):
    keys = non_conditional_df.columns[i]
    print(f"{keys} : {non_conditional_df[keys].mean()} ± {non_conditional_df[keys].var()}")

色鮮やか : 0.6973528148269329 ± 0.0003842479434375992
体にいい, 体によさそう : 0.6135178722480996 ± 0.00028789235854058854
ボリューム感がある : 0.5881736972198398 ± 0.0008578316810461184
珍しい : 0.5307356073830337 ± 0.004741880944941877
一口サイズ : 0.6357132194324571 ± 0.002675501413492435
早く食べれる : 0.42492061531037867 ± 0.0011356945204430203
場所が固まっている : 0.20271439136625355 ± 0.0023110056832147115


In [2]:
conditional_0 = {'色鮮やか': 0.7677892857446049, '体にいい, 体によさそう': 0.537122571679345, 'ボリューム感がある': 0.7025871277291539, '珍しい': 0.5425434509470823, '一口サイズ': 0.5935909781429091, '早く食べれる': 0.394609429972449, '場所が固まっている': 0.29588142587195304}
conditional_1 = {'色鮮やか': 0.7090677551858803, '体にいい, 体によさそう': 0.5960370162658636, 'ボリューム感がある': 0.6868293615592453, '珍しい': 0.5409549956369476, '一口サイズ': 0.6877761607205588, '早く食べれる': 0.35098008093874605, '場所が固まっている': 0.16371616317379623}
conditional_2 = {'色鮮やか': 0.687708880829605, '体にいい, 体によさそう': 0.5786696651960004, 'ボリューム感がある': 0.6748885828978286, '珍しい': 0.3912003715194349, '一口サイズ': 0.5680932984918483, '早く食べれる': 0.44745090651440583, '場所が固まっている': 0.2833968381140597}
conditional_3 = {'色鮮やか': 0.6764731444976682, '体にいい, 体によさそう': 0.5372979625880193, 'ボリューム感がある': 0.7019885580845195, '珍しい': 0.5957348999668446, '一口サイズ': 0.6710563745759611, '早く食べれる': 0.47653217793521196, '場所が固まっている': 0.16719313599818322}
conditional_4 = {'色鮮やか': 0.6981074646371574, '体にいい, 体によさそう': 0.5846908250871765, 'ボリューム感がある': 0.757951453339833, '珍しい': 0.5533596178136105, '一口サイズ': 0.5751862277115275, '早く食べれる': 0.4577007321770341, '場所が固まっている': 0.20593312938397765}
col = ["色鮮やか", "体にいい, 体によさそう", "ボリューム感がある", "珍しい", "一口サイズ", "早く食べれる", "場所が固まっている"]



In [16]:
corr_list = []
corr_list.append(conditional_0)
corr_list.append(conditional_1)
corr_list.append(conditional_2)
corr_list.append(conditional_3)
corr_list.append(conditional_4)


In [25]:
for factor_i in range(len(corr_list[0])):
    print(col[factor_i])
    cnt = 0
    for cv_num in range(len(corr_list)):
        cnt = cnt + corr_list[cv_num][col[factor_i]]
    print(cnt/len(corr_list))

色鮮やか
0.7078293061789831
体にいい, 体によさそう
0.566763608163281
ボリューム感がある
0.7048490167221161
珍しい
0.524758667176784
一口サイズ
0.6191406079285608
早く食べれる
0.4254546655075694
場所が固まっている
0.22322413850839395


In [26]:
for factor_i in range(1):#len(corr_list[0])):
    temp_list = []
    for cv_num in range(len(corr_list)):
        temp_list.append(corr_list[cv_num][col[factor_i]])

temp_list

[0.7677892857446049,
 0.7090677551858803,
 0.687708880829605,
 0.6764731444976682,
 0.6981074646371574]

In [29]:
np.mean(temp_list)

0.7078293061789831

In [3]:
conditional_0[col[0]]

0.7677892857446049

In [11]:
sum(conditional_0.values()) / len(conditional_0)

0.5477320385839282